In [1]:
## Importação das bibliotecas

import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

In [2]:
## Importação dos datasets

df = pd.read_csv(r"C:\TCC\Datasets\data_by_artist.csv", delimiter =',', encoding='latin-1')

In [3]:
## Elimina duplicatas

df.artists.str.strip()
df = df.drop_duplicates(subset='artists',keep=False)


## Dropa colunas não utilizadas

df.drop(['count','duration_ms','key','mode'],axis=1,inplace=True)


## Padroniza as colunas

x=df[df.drop(columns=['artists']).columns].values
scaler =StandardScaler().fit(x)
X_scaled = scaler.transform(x)
df[df.drop(columns=['artists']).columns]=X_scaled

In [4]:
## Cria as variáveis utilizadas na recomendação

df2=pd.melt(df,id_vars=['artists'],var_name='song_features', value_name='values')


## Cria uma tabela pivoteada do dataset

art_mat=df2.pivot_table(index="song_features",columns="artists",values="values")

In [5]:
## Inserindo os artistas favoritos:

def lista(): 
    artistas = []
    print("Insira o nome do artista analisado: ")
    inp = input()
    artistas.append(inp)
    
    while inp != "":
        print("Adicione mais artistas, se necessário: ")
        inp = input()
        artistas.append(inp)
    
    artistas.remove("")    
    print(f"A lista de artistas recomendados para {artistas}"" é:")
    return artistas

In [6]:
##Função de recomendação:

def recomenda(lista_artista,artist_matrix): 
    lista_artista = lista() 
    art_corr_df = pd.DataFrame()
    count = 0
    for artista in lista_artista:
        
        if artista in art_mat.columns:
            artist_values = artist_matrix[artista]
            similares = artist_matrix.corrwith(artist_values)
            similares = pd.DataFrame(similares,columns=['Correlation'])
            similares = similares.sort_values(by="Correlation",ascending=False).head(10)
            art_corr_df = art_corr_df.append(similares)
        
        else:
            print(f"{artista} não existe no dataset")
            count = count + 1
    
    if len(lista_artista)==count:        
        return print("Nenhum artista digitado está no dataset.")
    
    else:
        art_corr_df.dropna(inplace=True)
        art_corr_df.reset_index(inplace=True)
        art_corr_df = art_corr_df.drop_duplicates(subset='artists',keep = False)
        art_corr_df = art_corr_df[art_corr_df.Correlation != 1.000000]
        art_corr_df = art_corr_df.sort_values(by="Correlation",ascending = False)
        art_corr_df.reset_index(inplace=True)
        art_corr_df = art_corr_df.drop('index',axis=1)
        
        return art_corr_df

In [7]:
lista_artista = lista
recomenda(lista_artista, art_mat)

Insira o nome do artista analisado: 
Madonna
Adicione mais artistas, se necessário: 

A lista de artistas recomendados para ['Madonna'] é:


,artists,Correlation
0,Madonna,1.000000
1,Tina Turner,0.988023
2,Superpower,0.981694
3,Young-D,0.981694
4,Erick Morillo,0.981667
5,Technotronic,0.980418
6,Charly GarcÃ­a,0.979646
7,Los Amigos Invisibles,0.978197
8,Reel 2 Real,0.978077
9,After 7,0.977500
